# Optimisation: SNES

This example shows you how to run a global optimisation with SNES.

For a more elaborate example of an optimisation, see: https://github.com/martinjrobins/pints/blob/master/examples/optimisation-first-example.ipynb

In [3]:
from __future__ import print_function
import pints
import pints.toy as toy
import numpy as np
import matplotlib.pyplot as pl

# Load a forward model
model = toy.LogisticModel()

# Create some toy data
real_parameters = [0.015, 500]
times = np.linspace(0, 1000, 1000)
values = model.simulate(real_parameters, times)

# Create an object with links to the model and time series
problem = pints.SingleSeriesProblem(model, times, values)

# Select a score function
score = pints.SumOfSquaresError(problem)

# Select some boundaries
boundaries = pints.Boundaries([0, 400], [0.03, 600])

# Perform an optimization with boundaries and hints
x0 = 0.01, 450
sigma0 = [0.01, 100]
found_parameters, found_value = pints.snes(
    score,
    boundaries,
    x0,
    sigma0,
    )

print('Score at true solution: ')
print(score(real_parameters))

print('Found solution:          True parameters:' )
for k, x in enumerate(found_parameters):
    print(pints.strfloat(x) + '    ' + pints.strfloat(real_parameters[k]))

Running in parallel mode with population size 8
1: 1361453.12238
2: 1361453.12238
3: 1361453.12238
20: 609.452446271
40: 0.44194361517
60: 0.00481534539919
80: 2.00003625257e-05
100: 8.75809536184e-09
120: 1.79488297137e-11
140: 1.31736321615e-15
160: 5.23189909276e-17
180: 4.91524209761e-20
200: 4.38007681218e-23
220: 0.0
Halting: No significant change for 100 iterations.
222: 0.0
Score at true solution: 
0.0
Found solution:          True parameters:
 1.49999999999999994e-02     1.49999999999999994e-02
 5.00000000000000000e+02     5.00000000000000000e+02
